In [1]:
import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random
from BendLength import BendLengthCalculator
from preprocess import preProcessData
import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
point_cloud_location = "/home/chris/Code/PointClouds/data/FLIPscans/Bendy/Bendy_1/scan1_Part1.ply"
pcd = o3d.io.read_point_cloud(point_cloud_location)

mesh = o3d.io.read_triangle_mesh("/home/chris/Code/PointClouds/data/FLIPscans/Bendy/BendyCAD.STL")
mesh.compute_vertex_normals()

TriangleMesh with 4156 points and 2104 triangles.

In [ ]:
import open3d as o3d
import numpy as np

def visualize_main_plane_ransac(pcd, pt_to_plane_dist=0.5, plane_color=[0.3, 0.3, 1.0], rest_color=[0.8, 0.8, 0.8], visualize=True):
    """
    Segments the dominant plane using RANSAC and shows it colored differently from the rest.
    
    Parameters:
    - pcd: o3d.geometry.PointCloud, the input point cloud
    - pt_to_plane_dist: float, RANSAC distance threshold
    - plane_color: RGB list, color of the detected plane
    - rest_color: RGB list, color of the rest of the point cloud
    - visualize: bool, whether to display the result
    """
    # Plane segmentation
    plane_model, inliers = pcd.segment_plane(distance_threshold=pt_to_plane_dist, ransac_n=3, num_iterations=1000)

    # Create two subsets
    main_plane = pcd.select_by_index(inliers)
    rest = pcd.select_by_index(inliers, invert=True)

    # Color
    main_plane.paint_uniform_color(plane_color)
    rest.paint_uniform_color(rest_color)

    if visualize:
        o3d.visualization.draw_geometries([main_plane, rest])

    return main_plane, rest, plane_model

def visualize_dbscan_clusters(pcd, pt_to_plane_dist=0.4, eps=0.6, min_points=15, min_cluster_size=300):
    # Segment the main plane
    plane_model, inliers = pcd.segment_plane(distance_threshold=pt_to_plane_dist, ransac_n=3, num_iterations=1000)
    remaining = pcd.select_by_index(inliers, invert=True)

    # Step 1: Show remaining points in gray
    remaining.paint_uniform_color([0.6, 0.6, 0.6])
    o3d.visualization.draw_geometries([remaining], window_name="Remaining (Main Surface Removed)")

    # Step 2: Apply DBSCAN
    labels = np.array(remaining.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

    valid_indices = []
    valid_labels = []
    for label in np.unique(labels):
        if label == -1:
            continue  # skip noise
        label_indices = np.where(labels == label)[0]
        if len(label_indices) >= min_cluster_size:
            valid_indices.extend(label_indices)
            valid_labels.extend([label] * len(label_indices))

    if not valid_indices:
        print("No clusters found with the required size.")
        return

    clustered = remaining.select_by_index(valid_indices)
    valid_labels = np.array(valid_labels)
    max_label = valid_labels.max()
    colors = plt.get_cmap("tab20")(valid_labels / (max_label + 1))
    clustered.colors = o3d.utility.Vector3dVector(colors[:, :3])

    o3d.visualization.draw_geometries([clustered], window_name="DBSCAN Clusters (Filtered)")


In [4]:
# Preprocess the point cloud
pcd, average_density, cad_pcd = preProcessData(pcd, mesh, x_rotation=90, y_rotation=20, z_rotation=0)
pcd_points = np.asarray(pcd.points)
pcd_tree = o3d.geometry.KDTreeFlann(pcd)
# Detect planes, intersections, and anchor points
visualize_dbscan_clusters(pcd, pt_to_plane_dist=0.5)


Precompute neighbors.[========================================] 100%
